In [ ]:
import pandas as pd
from pandas.io import sql
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

In [ ]:
import sqlite3

In [ ]:
#This creates a database that we are connected to, but we have no data, at the moment this is empty.
conn = sqlite3.connect('dat-test.db')

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arugments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [ ]:
#We read in the data from CSV
data = pd.read_csv('dataset/rossmann_sales.csv', low_memory=False)
data.head()

In [ ]:
#This links the dataframe to our SQLlite database
data.to_sql('rossmann_sales',
            con=conn,
            if_exists='replace',
            index=False)

In [ ]:
sql.read_sql("select * from rossmann_sales where customers > 500 limit 10", con=conn)

> #### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database from it

In [ ]:
new_data = pd.read_csv('dataset/rossmann_sales.csv', low_memory=False)
new_data.to_sql("new_table", con=conn, if_exists='replace', index=False)

> #### CHECK: (SELECT) Have the students write a query that returns the `Store`, `Date` and `Customers`

In [ ]:
sql.read_sql("select store, date, customers from new_table limit 10", con=conn)

In [ ]:
sql.read_sql("select store, date, sales, customers from new_table where sales > 0 group by date limit 25", con=conn)

> #### CHECK: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [ ]:
sql.read_sql("select store, date, customers from new_table where open = 1 and promo = 1 group by store", con=conn)

> #### CHECK: (GROUP BY) Have the students write a query that returns the total sales on promotion days.

In [ ]:
sql.read_sql("select sum(sales) from new_table where promo = 1", con=conn)

#### Exercises

1. Load the Walmart sales and store features data
1. Create a table for each of those datasets
1. Select the store, date and fuel price on days it was over 90 degrees
1. Select the store, date and weekly sales and temperature
1. What were average sales on holiday vs. non-holiday sales
1. What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

In [ ]:
#Loading the data
wal_conn = sqlite3.connect('wal-mart.db')
walmart = pd.read_csv('dataset/walmart-sales.csv', low_memory=False)
#Table for data
walmart.to_sql('waldata',
            con=wal_conn,
            if_exists='replace',
            index=False)
#Table for features
walfeatures = pd.read_csv('dataset/features.csv', low_memory=False)
walfeatures.to_sql('walfeatures',
            con=wal_conn,
            if_exists='replace',
            index=False)

In [ ]:
sql.read_sql("select * from walfeatures limit 5", con=wal_conn)

In [ ]:
sql.read_sql("select * from waldata limit 5", con=wal_conn)

In [ ]:
#Select the store, date and fuel price on days it was over 90 degrees
sql.read_sql("select s.store, s.date, f.fuel_price from waldata as s join walfeatures as f on s.store = f.store "\
             "and s.date = f.date where temperature>90 limit 10", con=wal_conn)

In [ ]:
#Select the store, date and weekly sales and temperature
sql.read_sql("select s.store, s.date, s.weekly_sales, f.temperature from waldata as s join walfeatures as f "\
             "on s.store = f.store and s.date = f.date limit 10",con=wal_conn)

In [ ]:
#What were average sales on holiday vs. non-holiday sales
sql.read_sql("select isholiday, avg(weekly_sales) from waldata group by IsHoliday",con=wal_conn)

In [ ]:
#What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees
sql.read_sql("select s.isholiday, avg(s.weekly_sales) from waldata as s join walfeatures as f "\
             "on s.date = f.date and s.store = f.store where f.temperature<32 group by s.IsHoliday",con=wal_conn)

In [ ]:
#Unemployment
unemp = sql.read_sql("select s.date, s.weekly_sales, f.CPI, f.unemployment from waldata as s join walfeatures as f "\
             "on s.date = f.date and s.store = f.store group by s.date limit 100",con=wal_conn)
unemp = pd.DataFrame(unemp)

In [ ]:
unemp.head()

In [ ]:
unemp.plot.scatter(x="Weekly_Sales",y="Unemployment", s=1.5**unemp["CPI"])